In [1]:
from bs4 import BeautifulSoup as bs
import re
import requests
import pandas as pd
import time

In [2]:
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',200)

In [3]:
data_all = pd.read_csv('./1090531_test_try_catch.csv') #2004~2020每場比賽
#data_all['隊伍'].value_counts()   #誠泰 5955 #米迪亞 1467 不管他們 #中信也不要， 共26769

In [4]:
#data_all

In [5]:
del data_all['Unnamed: 0'] 
#刪掉三隊
data_all = data_all[(~data_all['主隊'].isin(['中信','誠泰','米迪亞'])) & (~data_all['客隊'].isin(['中信','誠泰','米迪亞']))]
#刪掉投手與非先發
data_all = data_all[(data_all['守備位置'] != 'P')&(~data_all['守備位置'].str.startswith(' ('))]
#data_all['隊伍'].value_counts() 

In [6]:
#data_all['守備位置'].value_counts() 
#data_all

In [7]:
Lamigo = data_all[(data_all['隊伍'].isin(['Lamigo','樂天']))] #做lamigo
ID_lst = list(set(Lamigo['球員ID'].values))
print(len(set(Lamigo['球員ID'].values)))
type_ = 1 ######################打者習慣#################
df_total = pd.DataFrame()
num = 0
for ID in ID_lst:
    print('現在到：'+ str(num))
    url = 'http://www.cpbl.com.tw/players/apart.html?player_id={}'.format(ID)
    time.sleep(2)
    re = requests.get(url) 
    soup = bs(re.text, 'html.parser')
    year_lst = [option.text for option in soup.find_all("select",{"class":"date_select"})[1].find_all("option")]
    for year in year_lst:
        response = requests.get(url + f'&year={year}'+f'&type=0{type_}')
        time.sleep(2)
        print(url + f'&year={year}'+f'&type=0{type_}')
        if response.status_code != 200:
            print(print('URL發生錯誤：' + url))
        soup = bs(response.text, 'html.parser')
        table = soup.find_all('tr')
        if len(table)>3:
            #變數名稱
            title_list = table[2].text.split('\n')
            title_list[0:2] = ['num','R/L']
            df = pd.DataFrame(columns = title_list)
            for i in range(3,len(table)):
                df.loc[i] = table[i].text.split('\n')
            df['year'] = year
            df['ID'] = ID
            df['team'] = soup.find('div','h1_side').a['href'].replace('/web/team_brief.php?&team=','')
            df_total = pd.concat([df_total,df])
        response.close()
    num +=1
df_total

94
現在到：0
http://www.cpbl.com.tw/players/apart.html?player_id=R206&year=2016&type=01
http://www.cpbl.com.tw/players/apart.html?player_id=R206&year=2015&type=01


ConnectionError: HTTPConnectionPool(host='www.cpbl.com.tw', port=80): Max retries exceeded with url: /players/apart.html?player_id=R206&year=2014&type=01 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022671859348>: Failed to establish a new connection: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。'))

In [ ]:
df_total.to_csv('./RL_type1.csv',encoding='utf-8',index=False)